In [4]:
from skimage.metrics import structural_similarity as ssim
import pandas as pt
import numpy as np
import os
import cv2

In [ ]:
## Code from GPT

from moviepy.editor import VideoFileClip
import math

# Load the video file
video_path = '/mnt/data/StatQuest - PCA main ideas in only 5 minutes!!!-HMOI_lkzW08.mp4'
clip = VideoFileClip(video_path)

# Calculate the total number of frames and the duration of the video
total_frames = math.ceil(clip.fps * clip.duration)
duration = clip.duration

total_frames, duration

###################################################################################################

from skimage.metrics import structural_similarity as ssim
import numpy as np
import cv2

def extract_significant_frames(video_path, threshold=0.02):
    """
    Extract frames from the video that have significant differences from each other.
    A frame is considered significant if the structural similarity index (SSIM) between it and the
    previous significant frame is less than 1 - threshold.

    Args:
    - video_path: Path to the video file.
    - threshold: The threshold for difference to consider a frame significant.

    Returns:
    - significant_frames: A list of tuples, where each tuple contains the frame number and the image data of a significant frame.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    significant_frames = []
    last_frame = None
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if last_frame is not None:
            # Convert frames to grayscale for SSIM calculation
            grayA = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
            grayB = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Calculate SSIM between current frame and last significant frame
            s = ssim(grayA, grayB)

            # If difference is significant, add to list
            if s < (1 - threshold):
                significant_frames.append((frame_idx, frame))
                last_frame = frame
        else:
            # First frame is always considered significant
            significant_frames.append((frame_idx, frame))
            last_frame = frame

        frame_idx += 1

    cap.release()
    return significant_frames

# Setting a threshold of 2% difference for significant change
significant_frames = extract_significant_frames(video_path, threshold=0.02)

len(significant_frames)  # Number of significant frames extracted

###################################################################################################

def extract_key_frames(video_path, skip_frames=5, threshold=0.02):
    """
    Extract key frames from the video, skipping a fixed number of frames between checks to improve performance.
    Frames are considered key if the SSIM difference with the previous key frame is above the specified threshold.
    
    Args:
    - video_path (str): Path to the video file.
    - skip_frames (int): Number of frames to skip between checks to reduce computation.
    - threshold (float): SSIM difference threshold to consider a frame as key.
    
    Returns:
    - List of tuples containing (frame_index, frame_data) for each key frame.
    """
    cap = cv2.VideoCapture(video_path)
    last_frame = None
    key_frames = []
    frame_idx = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video
        
        # Process every skip_frames-th frame for efficiency
        if frame_idx % skip_frames == 0:
            if last_frame is not None:
                grayA = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)
                grayB = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                s = ssim(grayA, grayB)

                if s < (1 - threshold):
                    key_frames.append((frame_idx, frame))
                    last_frame = frame
            else:
                # Always include the first frame
                key_frames.append((frame_idx, frame))
                last_frame = frame
        
        frame_idx += 1
    
    cap.release()
    return key_frames

# Attempting a more efficient approach by skipping frames
key_frames = extract_key_frames(video_path, skip_frames=30, threshold=0.02)

len(key_frames)  # Checking how many key frames were found this time

###################################################################################################

def extract_frames_at_intervals(video_path, interval=5):
    """
    Extract frames from the video at fixed intervals to reduce the total number of frames.
    
    Args:
    - video_path (str): Path to the video file.
    - interval (int): Interval in seconds between each frame to extract.
    
    Returns:
    - List of tuples containing (frame_index, frame_data) for each extracted frame.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames_to_extract = int(fps) * interval
    extracted_frames = []
    frame_idx = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video
        
        if frame_idx % frames_to_extract == 0:
            extracted_frames.append((frame_idx, frame))
        
        frame_idx += 1
    
    cap.release()
    return extracted_frames

# Extracting frames at 5-second intervals to efficiently capture key moments
interval_frames = extract_frames_at_intervals(video_path, interval=5)

len(interval_frames), interval_frames[0][0]  # Number of frames extracted and the index of the first extracted frame

###################################################################################################

from fpdf import FPDF

# Create instance of FPDF class
pdf = FPDF()

# Define PDF properties
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

# Loop through each frame and add it to the PDF
for idx, (frame_index, frame_data) in enumerate(interval_frames):
    # Save frame as image
    frame_path = f"/mnt/data/frame_{idx}.jpg"
    cv2.imwrite(frame_path, frame_data)
    
    # Add image to PDF
    pdf.add_page()
    pdf.image(frame_path, x=10, y=8, w=190)

# Save the PDF
pdf_output_path = "/mnt/data/extracted_frames.pdf"
pdf.output(pdf_output_path)

pdf_output_path



In [5]:
cap = cv2.VideoCapture('video.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)

In [6]:
fps

29.97002997002997

In [2]:
vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()

In [5]:
success, prev_frame = vidcap.read()

In [7]:
## Converting frames to grayscale
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
frame_diffs = []
saved_frames = []


In [10]:
saved_path = 'extracted_frames'
os.makedirs(saved_path, exist_ok=True)

In [9]:
############## Working code ####################



## Open the video file
vidcap = cv2.VideoCapture('video.mp4')
success, prev_frame = vidcap.read()
if not success:
    print("Failed to open video file or video file is empty.")

# Convert the first frame to grayscale
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
frame_diffs = []
saved_frames = []

# Path for saving extracted frames
save_path = 'extracted_frames'
os.makedirs(save_path, exist_ok=True)

frame_count = 0
while success:
    # Read next frame
    success, curr_frame = vidcap.read()
    if not success:
        break
    
    # Convert current frame to grayscale
    curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the absolute difference between current and previous frame
    diff = cv2.absdiff(curr_frame_gray, prev_frame_gray)
    non_zero_count = np.sum(diff > 25)  # Count of pixels that have changed

    # If significant change is detected, save the frame
    if (non_zero_count / diff.size) > 0.005:
        saved_frame_path = f"{save_path}/frame_{frame_count}.png"
        cv2.imwrite(saved_frame_path, curr_frame)
        saved_frames.append(saved_frame_path)
        prev_frame_gray = curr_frame_gray  # Update the previous frame

    frame_count += 1

# Clean up and release the video capture
vidcap.release()


#### Making PDF

In [10]:
from PIL import Image

In [12]:
images = [Image.open(frame).convert('RGB') for frame in saved_frames]

In [14]:
images[0].save('./test.pdf', save_all=True, append_images=images[1:])